In [12]:
import numpy as np
from copy import deepcopy

def sigmoid(z):
    """
    actiation function
    """
    return 1/(1+np.exp(-z))

def forward(X,W1,W2):
    """
    forward prop
    """
    z2 = np.dot(X, W1)
    a2 = sigmoid(z2)
    z3 = np.dot(a2, W2)
    yHat = sigmoid(z3) 
    return [z2,a2,z3,yHat]

def sigmoidPrime(z):
    """
    grad activation function
    """
    return np.exp(-z)/((1+np.exp(-z))**2)

def costFunction(X, y, W1, W2):
    """
    cost/loss function
    """
    fp = forward(X,W1,W2)
    yHat = fp[-1]
    J = 0.5*sum((y-yHat)**2)
    return fp,J

def costFunctionPrime(X, y, fp):
    """
    backprop
    """
    z2,a2,z3,yHat = fp
    delta3 = np.multiply(-(y-yHat), sigmoidPrime(z3))
    dJdW2 = np.dot(a2.T, delta3)
    delta2 = np.dot(delta3, W2.T)*sigmoidPrime(z2)
    dJdW1 = np.dot(X.T, delta2)  
    return dJdW1, dJdW2

# input, target, weights
X = np.array([[0.3, 1. ],
              [0.5 ,0.2],
              [1.  ,0.4]]) 
Y = np.array([[0.75],
              [0.82],
              [0.93]])
"""
Note that these weights have already been trained over, which explains the small L2 norm. 
"""
W1 = np.array([[ 1.62434536, -0.61175641 ,-0.52817175],
               [-1.07296862 , 0.86540763 ,-2.3015387 ]]) 
W2 = np.array([[ 1.74481176],
               [-0.7612069 ],
               [ 0.3190391 ]])

# forward training + loss
fp,loss = costFunction(X, Y, W1, W2)

# backward training (get gradients)
dW1, dW2 = costFunctionPrime(X, Y, fp)

# vectorize gradients (analytic)
ANALYTICALgrad = np.concatenate((dW1.ravel(),dW2.ravel()))

# vectorize parameters 
paramsInitial = np.concatenate((W1.ravel(),W2.ravel()))

# array for numerical gradients
NUMERICALgrad = np.zeros(paramsInitial.shape)

# perturbation array +/- epsilon
perturbs = np.zeros(paramsInitial.shape)

# arrays for perturbed parameters
paramsPlus = deepcopy(paramsInitial)
paramsMinus= deepcopy(paramsInitial)

# perturbation constant
epsilon = 1E-4

for weight in range(len(paramsInitial)):

    """
    After vectorizing all weights, concatenate them into a single vector. Perturb by + epsilon each weight,  
    then reshape all parameters back to their initial size. Do the same by pertube by - epsilon.
    With only that single weight perturbed forward and backward, run both through NN and calculate the
    two losses corresponding to the different pertubations. After obatining the
    losses, calculate the numerical gradient for that weight. Reset the weight back to its original
    value, then move on to the next weight and repeat. 
    """

    perturbs[weight] = epsilon
        
    ### Add small perturbation to each weight one at a time, then train the NN and get the loss   
    paramsPlus[weight] += perturbs[weight]
    ### reshape 
    w1_start = 0 
    w1_end =  w1_start + W1.shape[0] * W1.shape[1]
    W1 = np.reshape(paramsPlus[w1_start:w1_end], W1.shape)
    w2_start = w1_end
    w2_end = w2_start + W2.shape[0] * W2.shape[1]
    W2 = np.reshape(paramsPlus[w2_start:w2_end], W2.shape)    
    _ ,loss2 = costFunction(X, Y, W1, W2) # forward pass, get loss
    paramsPlus[weight] -= perturbs[weight] # reset back to original
    
    ### Subtract small perturbation to each weight one at a time, then train the NN and get the loss 
    paramsMinus[weight] -= perturbs[weight]
    ### reshape
    w1_start = 0 
    w1_end =  w1_start + W1.shape[0] * W1.shape[1]
    W1 = np.reshape(paramsMinus[w1_start:w1_end], W1.shape)
    w2_start = w1_end
    w2_end = w2_start + W2.shape[0] * W2.shape[1]
    W2 = np.reshape(paramsMinus[w2_start:w2_end], W2.shape)
    _ ,loss1 = costFunction(X, Y, W1, W2) # forward pass, get loss    
    paramsMinus[weight] += perturbs[weight] # reset back to original
    
    ### compute the numerical gradient
    NUMERICALgrad[weight] = (loss2 - loss1)/(2*epsilon)

# L2 Norm between analytic and numerical
np.linalg.norm(ANALYTICALgrad-NUMERICALgrad)/np.linalg.norm(ANALYTICALgrad+NUMERICALgrad)

3.419011823876603e-10